In [2]:
# https://stackoverflow.com/questions/44914887/stanford-pos-tagger-with-nltk-in-arabic-text
# https://textminingonline.com/dive-into-nltk-part-v-using-stanford-text-analysis-tools-in-python
# https://stackoverflow.com/questions/44907446/pos-tagging-arabic-text-using-nltk-in-python

# https://nlp.stanford.edu/~manning/papers/tagging.pdf
# http://www.nltk.org/api/nltk.tag.html#module-nltk.tag.stanford
# http://www.nltk.org/_modules/nltk/tag/stanford.html

# import os
# java_path = "/usr/bin/java/javapath/java.exe"
# os.environ['JAVAHOME'] = java_path

from nltk import StanfordPOSTagger
# from nltk.tag.stanford import StanfordTagger

path_to_model = ('/Users/zeinab/Desktop/Arabic Adjective Ordering Corpus Analysis/stanford-postagger-full-2018-10-16/models/arabic.tagger')

path_to_jar = ('/Users/zeinab/Desktop/Arabic Adjective Ordering Corpus Analysis/stanford-postagger-full-2018-10-16/stanford-postagger.jar')

artagger = StanfordPOSTagger(path_to_model, path_to_jar, encoding='utf8')

artagger._SEPARATOR = "/"

# tagged_sent = artagger.tag(". شرب القط الحليب اللذيذ".split())

# tagged_sent = artagger.tag("وجدت طابة دائريةً زرقاء على الطاولة الخشبية البنية الصغيرة. أخذتها و خرجت إلى الحديقة. لعبت فيها أنا وأخي الصغير. كانت الورود الحمراء تملأ الحديقة الخضراء الجميلة.   ".split())

In [3]:
simple_ex = "كانت الورود الحمراء تملأ الحديقة الخضراء الجميلة"

simple_ex_list = simple_ex.split()
# print(simple_ex_list)

simple_ex_tagged = artagger.tag(simple_ex_list)
print(simple_ex_tagged)

# I want to see how many (DTNN, DTJJ, DTJJ) or (NN,JJ,JJ) sequences there are.
# The possible adjective adjective noun phrases:
# - NN, JJ, JJ
# - DTNN, DTJJ, DTJJ
# - DTNN, JJ, JJ (SHOULDN'T GARAMATICALLY)
# - DTNN, DTJJ, JJ (SHOULDN'T GARAMATICALLY)
# We are looking for 3-grams with the first from DTNN or NN and the second and third from DTJJ or JJ.
# I want a function that takes a string as an input and outputs the 3-grams described above stored in a list 
# without their POS.

sub_simple_ex_tagged = simple_ex_tagged[4:7]

[('كانت', 'VBD'), ('الورود', 'DTNN'), ('الحمراء', 'DTJJ'), ('تملأ', 'VBP'), ('الحديقة', 'DTNN'), ('الخضراء', 'DTJJ'), ('الجميلة', 'DTJJ')]


In [7]:
# s = 'bla. bla? bla.bla! bla...'
# import re
# s = re.sub('([.,!?()])', r' \1 ', s)
# s = re.sub('\s{2,}', ' ', s)
# print(s)

bla . bla ? bla . bla ! bla . . . 


In [4]:
import re

def PunctuationPadder(s):
    """
    Input
    -----
    BLA BLA
    
    Output
    ------
    ZBLA BLA
    """
    s = re.sub('([.,!?()])', r' \1 ', s)
    s = re.sub('\s{2,}', ' ', s)
    return s

In [9]:
passage = "وجدت طابة دائريةً زرقاء على الطاولة الخشبية البنية الصغيرة. أخذتها و خرجت إلى الحديقة. لعبت فيها أنا وأخي الصغير. كانت الورود الحمراء تملأ الحديقة الخضراء الجميلة.   "
passage = PunctuationPadder(passage)
artagger.tag(passage.split())

[('وجدت', 'VBD'),
 ('طابة', 'NN'),
 ('دائريةً', 'JJ'),
 ('زرقاء', 'JJ'),
 ('على', 'IN'),
 ('الطاولة', 'DTNN'),
 ('الخشبية', 'DTJJ'),
 ('البنية', 'DTNN'),
 ('الصغيرة', 'DTJJ'),
 ('.', 'PUNC'),
 ('أخذتها', 'NN'),
 ('و', 'CC'),
 ('خرجت', 'VBD'),
 ('إلى', 'NN'),
 ('الحديقة', 'DTNN'),
 ('.', 'PUNC'),
 ('لعبت', 'VBD'),
 ('فيها', 'NN'),
 ('أنا', 'VBD'),
 ('وأخي', 'NNP'),
 ('الصغير', 'DTJJ'),
 ('.', 'PUNC'),
 ('كانت', 'VBD'),
 ('الورود', 'DTNN'),
 ('الحمراء', 'DTJJ'),
 ('تملأ', 'VBP'),
 ('الحديقة', 'DTNN'),
 ('الخضراء', 'DTJJ'),
 ('الجميلة', 'DTJJ'),
 ('.', 'PUNC')]

In [5]:
def NounAdjAdjChecker(sub_simple_ex_tagged):
    """
    Input
    -----
    sub_simple_ex_tagged is a list of 3 tuples and each tuple contains the word and POS
    e.g. sub_simple_ex_tagged = [('الحديقة', 'DTNN'), ('الخضراء', 'DTJJ'), ('الجميلة', 'DTJJ')]
    
    OutputAZ
    ------
    boolean: True iff we have noun adj adj string
    """

    # check if the 0-th word is a noun or not
    check_0 = sub_simple_ex_tagged[0][1] in ['DTNN', 'NN']

    # check if the 1-th word is a adjective or not
    check_1 = sub_simple_ex_tagged[1][1] in ['DTJJ', 'JJ']

    # check if the 2-th word is a adjective or not
    check_2 = sub_simple_ex_tagged[2][1] in ['DTJJ', 'JJ']
    
    full_check = check_0 and check_1 and check_2
    
    word_0 = sub_simple_ex_tagged[0][0]
    
    word_1 = sub_simple_ex_tagged[1][0]
    
    word_2 = sub_simple_ex_tagged[2][0]
    
    phrase = word_0 + "," + word_1 + "," + word_2 
    
    return full_check, phrase


In [19]:
# sub_simple_ex_tagged = [('وجدت', 'VBD'), ('طابة', 'NN'), ('دائريةً', 'JJ')]

# check_0 = sub_simple_ex_tagged[0][1] in ['DTNN', 'NN']

# check_1 = sub_simple_ex_tagged[1][1] in ['DTJJ', 'JJ']

# check_2 = sub_simple_ex_tagged[2][1] in ['DTJJ', 'JJ']

# noun = 

# if sub_simple_ex_tagged[0][1] == 'DTNN':
#     sub_simple_ex_tagged[1][1] and sub_simple_ex_tagged[2][1] == 'DTJJ'

# else








# full_check = check_0 and check_1 and check_2

# if check_0 = 'DTNN'
# check_1 = 'DTJJ' and check_2 = 'DTJJ'

# if check_0 = 'NN'
# check_1 = 'JJ' and check_2 = 'JJ'


SyntaxError: invalid syntax (<ipython-input-19-462ae67c4685>, line 9)

In [ ]:
if 1 == 1:
    print('Yes, 1=1')
else:
    print('Not')

In [13]:
sub_simple_ex_tagged = [('وجدت', 'VBD'), ('طابة', 'NN'), ('دائريةً', 'JJ')]

# check if the 0-th word is a noun or not
check_0 = sub_simple_ex_tagged[0][1] in ['DTNN', 'NN']

# check if the 1-th word is a adjective or not
check_1 = sub_simple_ex_tagged[1][1] in ['DTJJ', 'JJ']

# check if the 2-th word is a adjective or not
check_2 = sub_simple_ex_tagged[2][1] in ['DTJJ', 'JJ']

full_check = check_0 and check_1 and check_2

word_0 = sub_simple_ex_tagged[0][0]

word_1 = sub_simple_ex_tagged[1][0]

word_2 = sub_simple_ex_tagged[2][0]

phrase = word_0 + "," + word_1 + "," + word_2 

In [15]:
full_check

False

In [16]:
phrase

'وجدت,طابة,دائريةً'

In [6]:
def NounAdjAdjGenerator(tagged_sent):
    """
    Input: 
    ------
    tagged sentence of any length; example: [('وجدت', 'VBD'), ('طابة', 'NN'), ('دائريةً', 'JJ'), ('زرقاء', 'JJ'), ('على', 'IN'), ('الطاولة', 'DTNN')]
    and iterates through the sentence in 3-grams. If the tagged sentence contains adjadjnouns, it append it to a list that eventually is returned. 
    
    Output:
    ------
    adjadjnoun phrases
    """
    adjadjnouns = []
    for n in range(3,len(tagged_sent)+1):
        check,phrase = NounAdjAdjChecker(tagged_sent[n-3:n])
        if check:
             adjadjnouns.append(phrase)
    return adjadjnouns
    

In [7]:
def NounAdjAdjParser(simple_ex):
    """
    Input:
    ------
    takes in any sentence.
    
    Output:
    ------
    returns the sentence only if it's an adjadjnoun phrase, otherwise returns an empty list.
    """
    
    simple_ex_list = simple_ex.split()

    simple_ex_tagged = artagger.tag(simple_ex_list)
    
    return NounAdjAdjGenerator(simple_ex_tagged)

In [ ]:
# lines = open('/Users/zeinab/Downloads/ar.2013_48.raw')

In [ ]:
# AdjAdjNounPhrases = open('/Users/zeinab/Desktop/Arabic Adjective Ordering Corpus Analysis/common_crawl_adj_adj_nouns_20200315_v2.csv','w')
# AdjAdjNounPhrases.write("Adj2,Adj1,Noun\n")
# for line in lines: 
#     NewPhrases = NounAdjAdjParser(line)
#     for phrase in NewPhrases:
#         AdjAdjNounPhrases.write(phrase+"\n")
# AdjAdjNounPhrases.close()

In [8]:
from tqdm.notebook import tqdm

file_path = 'common_crawl_adj_adj_nouns_20200315_v4.csv'

with open('/Users/zeinab/Downloads/ar.2013_48.raw' ,'r') as lines:
    with open(file_path,'w') as AdjAdjNounPhrases:
        AdjAdjNounPhrases.write("Noun,Adj1,Adj2\n")
        for line in tqdm(lines):
            line = PunctuationPadder(line)
            NewPhrases = NounAdjAdjParser(line)
            for phrase in NewPhrases:
                AdjAdjNounPhrases.write(phrase+"\n")

KeyboardInterrupt: 

In [13]:
import pandas as pd

In [30]:
adj_used = pd.read_csv('Arabic Adjectives Used in Experiment.txt', header=None)
adj_used.columns = ["adjectives"]
adj_used.head()

,adjectives
0,الأحمر
1,الأخضر
2,الأزرق
3,الأصفر
4,البنفسجي


In [31]:
adj_used.adjectives

0           الأحمر
1          الأخضر 
2           الأزرق
3           الأصفر
4         البنفسجي
5           البني 
6          الكبير 
7           الصغير
8           الضخم 
9          القصير 
10         الطويل 
11         الخشبي 
12     البلاستيكي 
13        المعدني 
14         الناعم 
15          الصلب 
16          الأملس
17         القديم 
18         الجديد 
19         الفاسد 
20         الطازج 
21          الجيد 
22          السيئ 
23       المستدير 
24         المربع 
25        الحمراء 
26        الخضراء 
27        الزرقاء 
28        الصفراء 
29      البنفسجية 
30         البنية 
31         الكبيرة
32        الصغيرة 
33         الضخمة 
34        القصيرة 
35        الطويلة 
36        الخشبية 
37    البلاستيكية 
38       المعدنية 
39        الناعمة 
40         الصلبة 
41        الملساء 
42        القديمة 
43        الجديدة 
44        الفاسدة 
45        الطازجة 
46         الجيدة 
47         السيئة 
48      المستديرة 
49         المربعة
Name: adjectives, dtype: object

In [44]:
adj_list = adj_used.adjectives.to_list() #converts column (a series) into a list
adj_list = [word.replace(" ","") for word in adj_list]
adj_list

['الأحمر',
 'الأخضر',
 'الأزرق',
 'الأصفر',
 'البنفسجي',
 'البني',
 'الكبير',
 'الصغير',
 'الضخم',
 'القصير',
 'الطويل',
 'الخشبي',
 'البلاستيكي',
 'المعدني',
 'الناعم',
 'الصلب',
 'الأملس',
 'القديم',
 'الجديد',
 'الفاسد',
 'الطازج',
 'الجيد',
 'السيئ',
 'المستدير',
 'المربع',
 'الحمراء',
 'الخضراء',
 'الزرقاء',
 'الصفراء',
 'البنفسجية',
 'البنية',
 'الكبيرة',
 'الصغيرة',
 'الضخمة',
 'القصيرة',
 'الطويلة',
 'الخشبية',
 'البلاستيكية',
 'المعدنية',
 'الناعمة',
 'الصلبة',
 'الملساء',
 'القديمة',
 'الجديدة',
 'الفاسدة',
 'الطازجة',
 'الجيدة',
 'السيئة',
 'المستديرة',
 'المربعة']

In [53]:
phrases_extracted = pd.read_csv('common_crawl_adj_adj_nouns_20200315_v4.csv')
phrases_extracted.columns = ["Noun","Adj1","Adj2"]
phrases_extracted.head()

,Noun,Adj1,Adj2
0,أتانا,بتحية,وسلام
1,القادة,بالتهنئة,الواجبة
2,طلاب,واساتذة,وعليراسهم
3,مصدر,للطاقة,الكهربائية
4,القوة,الدافعة,الكهربائية


In [57]:
phrases_extracted.Adj1 == 'واحد'

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13       True
14       True
15       True
16      False
17      False
18      False
19      False
20      False
21       True
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
3267    False
3268    False
3269    False
3270    False
3271    False
3272    False
3273    False
3274    False
3275    False
3276    False
3277    False
3278    False
3279    False
3280    False
3281    False
3282    False
3283    False
3284    False
3285    False
3286    False
3287    False
3288    False
3289    False
3290    False
3291    False
3292    False
3293    False
3294    False
3295    False
3296    False
Name: Adj1, Length: 3297, dtype: bool

In [58]:
phrases_extracted[phrases_extracted.Adj1 == 'واحد']

,Noun,Adj1,Adj2
13,فولت,واحد,ونصف
14,فولت,واحد,ونصف
15,مصدر,واحد,إمداده،
21,بمقدار,واحد,ونصف
128,فضولية,واحد,عصبي
918,جاب,واحد,وهكذا
1588,مؤلف,واحد,كامل
1681,محغوظ,واحد,منهم
1729,طوله,واحد,ملليمتر
1730,بروتون,واحد,والكترون


In [59]:
phrases_extracted.Adj1.isin(adj_list) 

0       False
1       False
2       False
3       False
4       False
5       False
6       False
7       False
8       False
9       False
10      False
11      False
12      False
13      False
14      False
15      False
16      False
17      False
18      False
19      False
20      False
21      False
22      False
23      False
24      False
25      False
26      False
27      False
28      False
29      False
        ...  
3267    False
3268    False
3269    False
3270    False
3271    False
3272    False
3273    False
3274    False
3275    False
3276    False
3277    False
3278    False
3279    False
3280    False
3281    False
3282    False
3283    False
3284    False
3285    False
3286    False
3287    False
3288    False
3289    False
3290    False
3291    False
3292    False
3293    False
3294    False
3295    False
3296    False
Name: Adj1, Length: 3297, dtype: bool

In [61]:
phrases_extracted[phrases_extracted.Adj1.isin(adj_list)]

,Noun,Adj1,Adj2
114,الدراهم,الصغيرة,السابقة
672,البقع,الزرقاء,والحمراء
765,الشعر,الجيد,الصحيح
1026,الحقائب,البلاستيكية,الشفافة
1035,ضوابط,الجديدة,الخاصة
1085,المدرب,الجديد,لمولودية
1277,والإفطارالفنادق,الصغيرة,الموجودة
1325,والإفطارالفنادق,الصغيرة,المعروضة
1341,والإفطارالفنادق,الصغيرة,المعروضة
1452,والإفطارالفنادق,الصغيرة,الموجودة


In [62]:
phrases_extracted[phrases_extracted.Adj2.isin(adj_list)]

,Noun,Adj1,Adj2
108,الإستعمار,الفرنسي,الطويل
166,الأجهزة,المنزلية,الصغيرة
167,الأجهزة,المنزلية,الصغيرة
177,موت,المناضل,الكبير
178,موت,المناضل,الكبير
190,الأديب,المصري,الكبير
574,الأوضاع,المالية,الجديدة
945,المسجد,الشرقي,الكبير
1086,المدارس,الكروية,الكبيرة
1197,الاذاعة,لمتابعة,الجديد


In [79]:
a = phrases_extracted[phrases_extracted.Adj1.isin(adj_list)]
b = phrases_extracted[phrases_extracted.Adj2.isin(adj_list)]
c = pd.concat([a,b]).reset_index(drop=True)
c

,Noun,Adj1,Adj2
0,الدراهم,الصغيرة,السابقة
1,البقع,الزرقاء,والحمراء
2,الشعر,الجيد,الصحيح
3,الحقائب,البلاستيكية,الشفافة
4,ضوابط,الجديدة,الخاصة
5,المدرب,الجديد,لمولودية
6,والإفطارالفنادق,الصغيرة,الموجودة
7,والإفطارالفنادق,الصغيرة,المعروضة
8,والإفطارالفنادق,الصغيرة,المعروضة
9,والإفطارالفنادق,الصغيرة,الموجودة


In [81]:
c.drop_duplicates().shape

(41, 3)